# 🧹 Python Q&A Dataset Cleaning Pipeline

## Purpose
Clean and optimize your 149K Q&A pairs for better chatbot accuracy!

1.Remove duplicates
2.Fix formatting issues
3.Remove empty/invalid entries
4.Standardize text
5.Remove noise
6.Validate Q&A pairs
7.Export clean dataset


---
## Step 1: Install & Import Libraries

In [ ]:
# Install required packages
!pip install pandas numpy nltk fuzzywuzzy python-Levenshtein -q

import pandas as pd
import numpy as np
import re
import nltk
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

print("All libraries imported successfully!")

✅ All libraries imported successfully!


---
## 📂 Step 2: Upload Your Dataset Files

**Upload these 3 files from your `train/` folder:**
1. `train.csv`
2. `Dataset_Python_Question_Answer.csv`
3. `Python Programming Questions Dataset.csv`

In [ ]:
# Upload files in Google Colab
from google.colab import files

print("Please upload your CSV files...")
uploaded = files.upload()

print("\nFiles uploaded successfully!")
print("Uploaded files:", list(uploaded.keys()))

📤 Please upload your CSV files...


---
## 📊 Step 3: Load & Combine Datasets

In [ ]:
# Load all datasets
def load_all_datasets():
    dfs = []

    # Try to load each file
    files_to_load = [
        'train.csv',
        'Dataset_Python_Question_Answer.csv',
        'Python Programming Questions Dataset.csv'
    ]

    for file in files_to_load:
        try:
            df = pd.read_csv(file, encoding='utf-8')
            print(f" Loaded {file}: {len(df):,} rows")

            # Standardize column names
            if 'Question' in df.columns and 'Answer' in df.columns:
                df = df[['Question', 'Answer']]
            elif 'question' in df.columns and 'answer' in df.columns:
                df = df.rename(columns={'question': 'Question', 'answer': 'Answer'})

            dfs.append(df)
        except Exception as e:
            print(f" Could not load {file}: {e}")

    # Combine all datasets
    if dfs:
        combined_df = pd.concat(dfs, ignore_index=True)
        print(f"\nTotal combined rows: {len(combined_df):,}")
        return combined_df
    else:
        print(" No datasets loaded!")
        return None

df = load_all_datasets()

# Show sample
if df is not None:
    print("\n📋 Sample data:")
    display(df.head())

✅ Loaded train.csv: 136,109 rows
✅ Loaded Dataset_Python_Question_Answer.csv: 419 rows
✅ Loaded Python Programming Questions Dataset.csv: 13,109 rows

🎉 Total combined rows: 149,637

📋 Sample data:


,Answer,Question,Instruction,Input,Output
0,"Yes, you can format the output text in Bash to...",How can I output bold text in Bash? I have a B...,NaN,NaN,NaN
1,"To install Python 3 on an AWS EC2 instance, yo...",How can I install Python 3 on an AWS EC2 insta...,NaN,NaN,NaN
2,You can achieve the desired time format using ...,How can I format the elapsed time from seconds...,NaN,NaN,NaN
3,Your current implementation is actually quite ...,I am trying to create a matrix of random numbe...,NaN,NaN,NaN
4,The use of 'self' in Python is quite different...,I am learning Python and have noticed extensiv...,NaN,NaN,NaN


---
## 🔍 Step 4: Data Quality Analysis

In [ ]:
def analyze_data_quality(df):
    print("" + "="*60)
    print("📊 DATA QUALITY ANALYSIS")
    print("="*60 + "")

    # Basic info
    print(f"\n1️⃣ Total Records: {len(df):,}")
    print(f"   Columns: {list(df.columns)}")

    # Missing values
    print("\n2️⃣ Missing Values:")
    missing = df.isnull().sum()
    for col, count in missing.items():
        if count > 0:
            print(f"   ❌ {col}: {count:,} ({count/len(df)*100:.2f}%)")
        else:
            print(f"   ✅ {col}: 0")

    # Empty strings
    print("\n3️⃣ Empty Strings:")
    empty_questions = (df['Question'].str.strip() == '').sum()
    empty_answers = (df['Answer'].str.strip() == '').sum()
    print(f"   Empty Questions: {empty_questions:,}")
    print(f"   Empty Answers: {empty_answers:,}")

    # Duplicates
    print("\n4️⃣ Duplicates:")
    total_dupes = df.duplicated().sum()
    question_dupes = df.duplicated(subset=['Question']).sum()
    print(f"   Exact duplicates: {total_dupes:,}")
    print(f"   Duplicate questions: {question_dupes:,}")

    # Length statistics
    print("\n5️⃣ Text Length Statistics:")
    df['q_len'] = df['Question'].str.len()
    df['a_len'] = df['Answer'].str.len()

    print(f"\n   Questions:")
    print(f"   - Min: {df['q_len'].min()}")
    print(f"   - Max: {df['q_len'].max()}")
    print(f"   - Mean: {df['q_len'].mean():.0f}")
    print(f"   - Very short (<10 chars): {(df['q_len'] < 10).sum():,}")
    print(f"   - Very long (>500 chars): {(df['q_len'] > 500).sum():,}")

    print(f"\n   Answers:")
    print(f"   - Min: {df['a_len'].min()}")
    print(f"   - Max: {df['a_len'].max()}")
    print(f"   - Mean: {df['a_len'].mean():.0f}")
    print(f"   - Very short (<10 chars): {(df['a_len'] < 10).sum():,}")
    print(f"   - Very long (>5000 chars): {(df['a_len'] > 5000).sum():,}")

    # Drop temp columns
    df.drop(['q_len', 'a_len'], axis=1, inplace=True)

    print("\n" + "="*60)

analyze_data_quality(df)

📊 DATA QUALITY ANALYSIS

1️⃣ Total Records: 149,637
   Columns: ['Answer', 'Question', 'Instruction', 'Input', 'Output']

2️⃣ Missing Values:
   ❌ Answer: 13,110 (8.76%)
   ❌ Question: 13,109 (8.76%)
   ❌ Instruction: 136,528 (91.24%)
   ❌ Input: 142,077 (94.95%)
   ❌ Output: 136,528 (91.24%)

3️⃣ Empty Strings:
   Empty Questions: 0
   Empty Answers: 0

4️⃣ Duplicates:
   Exact duplicates: 0
   Duplicate questions: 13,653

5️⃣ Text Length Statistics:

   Questions:
   - Min: 17.0
   - Max: 7710.0
   - Mean: 237
   - Very short (<10 chars): 0
   - Very long (>500 chars): 18,543

   Answers:
   - Min: 8.0
   - Max: 13327.0
   - Mean: 1279
   - Very short (<10 chars): 2
   - Very long (>5000 chars): 44



---
## 🧹 Step 5: Data Cleaning Operations

In [ ]:
def clean_text(text):
    """Clean individual text field"""
    if pd.isna(text) or not isinstance(text, str):
        return ""

    # Remove extra whitespace
    text = ' '.join(text.split())

    # Remove multiple punctuation
    text = re.sub(r'([!?.]){2,}', r'\1', text)

    # Fix spacing around punctuation
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)

    # Remove URLs (optional - uncomment if needed)
    # text = re.sub(r'http\S+|www\S+', '', text)

    # Remove HTML tags (if any)
    text = re.sub(r'<[^>]+>', '', text)

    # Remove excessive line breaks
    text = re.sub(r'\n{3,}', '\n\n', text)

    return text.strip()


def clean_dataset(df):
    """Main cleaning function"""
    print("🧹 Starting data cleaning...\n")
    initial_count = len(df)

    # 1. Remove null values
    print("1️⃣ Removing null values...")
    df = df.dropna(subset=['Question', 'Answer'])
    print(f"   Removed: {initial_count - len(df):,} rows")

    # 2. Clean text
    print("\n2️⃣ Cleaning text fields...")
    df['Question'] = df['Question'].apply(clean_text)
    df['Answer'] = df['Answer'].apply(clean_text)
    print("   ✅ Text cleaned")

    # 3. Remove empty strings
    print("\n3️⃣ Removing empty entries...")
    before = len(df)
    df = df[(df['Question'].str.len() > 0) & (df['Answer'].str.len() > 0)]
    print(f"   Removed: {before - len(df):,} rows")

    # 4. Remove very short questions (likely invalid)
    print("\n4️⃣ Removing very short questions (<10 chars)...")
    before = len(df)
    df = df[df['Question'].str.len() >= 10]
    print(f"   Removed: {before - len(df):,} rows")

    # 5. Remove very short answers (likely invalid)
    print("\n5️⃣ Removing very short answers (<5 chars)...")
    before = len(df)
    df = df[df['Answer'].str.len() >= 5]
    print(f"   Removed: {before - len(df):,} rows")

    # 6. Remove exact duplicates
    print("\n6️⃣ Removing exact duplicates...")
    before = len(df)
    df = df.drop_duplicates()
    print(f"   Removed: {before - len(df):,} rows")

    # 7. Remove duplicate questions (keep first)
    print("\n7️⃣ Removing duplicate questions...")
    before = len(df)
    df = df.drop_duplicates(subset=['Question'], keep='first')
    print(f"   Removed: {before - len(df):,} rows")

    # 8. Remove questions that are just punctuation or numbers
    print("\n8️⃣ Removing invalid questions...")
    before = len(df)
    df = df[df['Question'].str.contains('[a-zA-Z]', regex=True)]
    print(f"   Removed: {before - len(df):,} rows")

    # 9. Reset index
    df = df.reset_index(drop=True)

    print("\n" + "="*60)
    print(f"✅ CLEANING COMPLETE!")
    print(f"   Initial rows: {initial_count:,}")
    print(f"   Final rows: {len(df):,}")
    print(f"   Removed: {initial_count - len(df):,} ({(initial_count - len(df))/initial_count*100:.1f}%)")
    print("="*60 + "\n")

    return df

# Perform cleaning
df_clean = clean_dataset(df.copy())

🧹 Starting data cleaning...

1️⃣ Removing null values...
   Removed: 13,110 rows

2️⃣ Cleaning text fields...
   ✅ Text cleaned

3️⃣ Removing empty entries...
   Removed: 2 rows

4️⃣ Removing very short questions (<10 chars)...
   Removed: 0 rows

5️⃣ Removing very short answers (<5 chars)...
   Removed: 0 rows

6️⃣ Removing exact duplicates...
   Removed: 0 rows

7️⃣ Removing duplicate questions...
   Removed: 546 rows

8️⃣ Removing invalid questions...
   Removed: 0 rows

✅ CLEANING COMPLETE!
   Initial rows: 149,637
   Final rows: 135,979
   Removed: 13,658 (9.1%)



---
## 📈 Step 6: Quality Check After Cleaning

In [ ]:
print("\n🔍 QUALITY CHECK AFTER CLEANING:\n")
analyze_data_quality(df_clean)

# Show sample of cleaned data
print("\n📋 Sample of cleaned data:")
display(df_clean.head(10))


🔍 QUALITY CHECK AFTER CLEANING:

📊 DATA QUALITY ANALYSIS

1️⃣ Total Records: 135,979
   Columns: ['Answer', 'Question', 'Instruction', 'Input', 'Output']

2️⃣ Missing Values:
   ✅ Answer: 0
   ✅ Question: 0
   ❌ Instruction: 135,979 (100.00%)
   ❌ Input: 135,979 (100.00%)
   ❌ Output: 135,979 (100.00%)

3️⃣ Empty Strings:
   Empty Questions: 0
   Empty Answers: 0

4️⃣ Duplicates:
   Exact duplicates: 0
   Duplicate questions: 0

5️⃣ Text Length Statistics:

   Questions:
   - Min: 16
   - Max: 6396
   - Mean: 231
   - Very short (<10 chars): 0
   - Very long (>500 chars): 18,142

   Answers:
   - Min: 41
   - Max: 11851
   - Mean: 1205
   - Very short (<10 chars): 0
   - Very long (>5000 chars): 18


📋 Sample of cleaned data:


,Answer,Question,Instruction,Input,Output
0,"Yes, you can format the output text in Bash to...",How can I output bold text in Bash? I have a B...,NaN,NaN,NaN
1,"To install Python 3 on an AWS EC2 instance, yo...",How can I install Python 3 on an AWS EC2 insta...,NaN,NaN,NaN
2,You can achieve the desired time format using ...,How can I format the elapsed time from seconds...,NaN,NaN,NaN
3,Your current implementation is actually quite ...,I am trying to create a matrix of random numbe...,NaN,NaN,NaN
4,The use of 'self' in Python is quite different...,I am learning Python and have noticed extensiv...,NaN,NaN,NaN
5,Gradient clipping is a technique to prevent ex...,What is the correct method to perform gradient...,NaN,NaN,NaN
6,The error is due to pip trying to connect over...,I am a Python beginner and I'm trying to use p...,NaN,NaN,NaN
7,Fuzzy matching is possible with Python pandas ...,How can I perform a fuzzy match merge using Py...,NaN,NaN,NaN
8,The issue you're experiencing is due to the wa...,I am new to Flask and Flask-Restful and I'm ha...,NaN,NaN,NaN
9,"In sklearn, the Logistic Regression model is i...",How can I find the weight vector 'w' in Binary...,NaN,NaN,NaN


---
## 🎯 Step 7: Advanced Cleaning (Optional)

In [ ]:
def advanced_cleaning(df):
    """Additional cleaning operations"""
    print("🔧 Advanced cleaning operations...\n")
    initial = len(df)

    # 1. Remove questions with too many special characters
    print("1️⃣ Removing questions with excessive special characters...")
    before = len(df)
    df['special_ratio'] = df['Question'].apply(
        lambda x: len(re.findall(r'[^a-zA-Z0-9\s]', x)) / len(x) if len(x) > 0 else 0
    )
    df = df[df['special_ratio'] < 0.3]  # Less than 30% special chars
    df.drop('special_ratio', axis=1, inplace=True)
    print(f"   Removed: {before - len(df):,} rows")

    # 2. Standardize Python-related terms
    print("\n2️⃣ Standardizing Python terms...")
    replacements = {
        r'\bpython\s+3\b': 'Python 3',
        r'\bpython\s+2\b': 'Python 2',
        r'\bpython3\b': 'Python 3',
        r'\bpython2\b': 'Python 2',
        r'\bpy\b': 'Python'
    }

    for pattern, replacement in replacements.items():
        df['Question'] = df['Question'].str.replace(pattern, replacement, regex=True, case=False)
        df['Answer'] = df['Answer'].str.replace(pattern, replacement, regex=True, case=False)

    print("   ✅ Terms standardized")

    # 3. Remove questions/answers that are too similar (fuzzy duplicates)
    # This is expensive, so we'll skip it for large datasets
    if len(df) < 50000:
        print("\n3️⃣ Detecting near-duplicates (this may take time)...")
        from fuzzywuzzy import fuzz

        # Sample approach: check for very high similarity
        # (For full dataset, this would be too slow)
        print("   ⏭️ Skipping for large dataset (use smaller sample if needed)")
    else:
        print("\n3️⃣ Dataset too large for fuzzy duplicate detection")
        print("   💡 Tip: Create a smaller sample if needed")

    print(f"\n✅ Advanced cleaning complete! Total removed: {initial - len(df):,}")
    return df

# Apply advanced cleaning
df_clean = advanced_cleaning(df_clean)

🔧 Advanced cleaning operations...

1️⃣ Removing questions with excessive special characters...
   Removed: 39 rows

2️⃣ Standardizing Python terms...
   ✅ Terms standardized

3️⃣ Dataset too large for fuzzy duplicate detection
   💡 Tip: Create a smaller sample if needed

✅ Advanced cleaning complete! Total removed: 39


---
## 📊 Step 8: Final Statistics

In [ ]:
def show_final_stats(df):
    print("" + "="*60)
    print("📊 FINAL DATASET STATISTICS")
    print("="*60 + "")

    print(f"\n✅ Total Q&A pairs: {len(df):,}")
    print(f"\n📏 Question lengths:")
    print(f"   - Average: {df['Question'].str.len().mean():.0f} characters")
    print(f"   - Median: {df['Question'].str.len().median():.0f} characters")
    print(f"\n📏 Answer lengths:")
    print(f"   - Average: {df['Answer'].str.len().mean():.0f} characters")
    print(f"   - Median: {df['Answer'].str.len().median():.0f} characters")

    # Top question patterns
    print(f"\n🔝 Top question starters:")
    first_words = df['Question'].str.split().str[0].value_counts().head(10)
    for word, count in first_words.items():
        print(f"   - '{word}': {count:,} questions")

    print("\n" + "="*60)

show_final_stats(df_clean)

📊 FINAL DATASET STATISTICS

✅ Total Q&A pairs: 135,940

📏 Question lengths:
   - Average: 231 characters
   - Median: 110 characters

📏 Answer lengths:
   - Average: 1205 characters
   - Median: 1150 characters

🔝 Top question starters:
   - 'How': 55,591 questions
   - 'I': 26,793 questions
   - 'Can': 22,475 questions
   - 'What': 13,167 questions
   - 'Is': 9,434 questions
   - 'I'm': 4,010 questions
   - 'In': 2,608 questions
   - 'Why': 619 questions
   - 'I've': 180 questions
   - 'As': 157 questions



---
## 💾 Step 9: Export Cleaned Dataset

In [ ]:
# Save cleaned dataset
output_filename = 'cleaned_python_qa_dataset.csv'
df_clean.to_csv(output_filename, index=False, encoding='utf-8')

print(f"✅ Cleaned dataset saved as: {output_filename}")
print(f"   Total rows: {len(df_clean):,}")
print(f"   File size: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB (in memory)")

# Download file (for Google Colab)
print("\n📥 Downloading file...")
files.download(output_filename)
print("\n🎉 Done! Upload this file to your project's train/ folder!")

✅ Cleaned dataset saved as: cleaned_python_qa_dataset.csv
   Total rows: 135,940
   File size: 216.11 MB (in memory)

📥 Downloading file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Done! Upload this file to your project's train/ folder!


---
## 📋 Step 10: Create Sample Dataset (For Testing)

In [ ]:
# Create a smaller sample for testing
sample_size = 10000  # Adjust as needed

if len(df_clean) > sample_size:
    df_sample = df_clean.sample(n=sample_size, random_state=42)
    sample_filename = f'sample_{sample_size}_qa_dataset.csv'
    df_sample.to_csv(sample_filename, index=False, encoding='utf-8')

    print(f"✅ Sample dataset created: {sample_filename}")
    print(f"   Rows: {len(df_sample):,}")
    print("\n📥 Downloading sample file...")
    files.download(sample_filename)
    print("\n💡 Use this for faster testing!")
else:
    print("Dataset is already small enough, no sample needed!")

✅ Sample dataset created: sample_10000_qa_dataset.csv
   Rows: 10,000

📥 Downloading sample file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


💡 Use this for faster testing!
